# Solving the Rastrigin problem using pgpelib

This example notebook demonstrates the core usage of `pgpelib` by solving the Rastrigin problem ([Rastrigin, 1974; Rudolph, 1990](#References)) using PGPE ([Sehnke et al., 2010](#References)) and the ClipUp optimizer ([Toklu et al., 2020](#References)) (or alternatively, the Adam optimizer ([Kingma et al., 2015](#References))).

## Initialization

We begin by importing the necessary libraries.

In [1]:
from pgpelib import PGPE
import numpy as np
from numbers import Real

Now, we define an n-dimensional Rastrigin function.

In [2]:
A = 10.0
n = 100     # <-- dimension

def rastrigin_function(x: np.ndarray) -> Real:
    cost = A * n + np.sum(x ** 2.0 - (A * np.cos(2 * np.pi * x)))
    # This cost value is to be minimized.
    # Because pgpelib assumes that the goal is to maximize,
    # we return the cost multiplied by -1.
    return -cost

We also generate an initial solution and name it `x0`:

In [3]:
x0 = np.random.uniform(-5.12, 5.12, n).astype('float32')
x0

array([ 2.260581  , -4.287902  ,  4.2492113 , -3.375462  ,  1.8905883 ,
        2.4019365 ,  2.3786597 ,  2.0572114 ,  4.2870283 , -5.0148764 ,
       -1.7789336 , -1.2420483 ,  0.57446414, -4.6260176 ,  4.804072  ,
       -4.6377134 ,  4.130985  , -2.1806664 , -0.84947735, -4.4363756 ,
       -1.3713521 ,  0.10298141, -3.453057  , -3.333743  ,  1.3536777 ,
        4.9601727 , -2.2064102 , -3.9028325 , -3.578574  ,  1.6291565 ,
       -3.1885226 , -3.2866375 ,  1.8182292 , -4.79836   ,  2.2121603 ,
        5.034507  , -0.6028644 ,  1.857549  ,  4.2413106 , -2.6413257 ,
        2.040794  , -3.7954023 , -4.675698  , -0.49074742,  0.7865207 ,
        3.7415774 ,  0.43679133, -1.6776841 , -4.83935   ,  0.23159784,
       -4.2978    , -0.6832779 , -1.425045  , -2.4312851 ,  4.7246075 ,
       -2.0583162 ,  0.5828159 ,  4.2087646 ,  0.48544732,  2.7085035 ,
       -1.5704492 , -1.6617717 ,  1.1273117 ,  1.9748904 , -1.4124613 ,
        1.8874893 ,  0.18855312, -5.0623074 ,  2.8473904 , -4.18

Now, let us instantiate a PGPE solver using the ClipUp optimizer.

In [4]:
pgpe = PGPE(

    # Length of each solution:
    solution_length=n,
    
    # Population size:
    popsize=250,
    
    # Initial center solution (i.e. initial mean of the search
    # distribution):
    center_init=x0,
    
    # Learning rate for when updating the mean of the search distribution:
    center_learning_rate=0.03,

    # The following configuration tells that the ClipUp optimizer
    # is to be used when updating the mean of the search distribution.
    optimizer='clipup',
    
    # The ClipUp-specific 'max_speed' hyperparameter is set here:
    optimizer_config={'max_speed': 0.06},
    
    # If instead of ClipUp you would like to use Adam,
    # then set:
    #     optimizer='adam'
    # and do not specify any optimizer_config so that Adam is used
    # with its default hyperparameters.
    # For customizing how Adam works, you can also set
    # optimizer_config={'beta1': ..., 'beta2': ..., 'epsilon': ...}
    
    # Initial standard deviation of the search distribution
    stdev_init=1.0,
    
    # Learning rate for when updating the standard deviation of the
    # search distribution:
    stdev_learning_rate=0.1,
    
    # With the setting below, the standard deviation cannot change
    # more tha then 20% of its original value.
    stdev_max_change=0.2,
    
    # When estimating the gradient, rank the solutions linearly
    # (the ranks go from -0.5 to 0.5 from worst to best, like done
    # in the work of Salimans et al. (2017)):
    solution_ranking=True,
    
    # The PGPE solver will work with arrays of the following dtype:
    dtype='float32'
)

pgpe

## Evolution loop

We are now ready to implement and execute our evolution loop.
Inspired by the existing evolutionary computation libraries of [Hansen et al. 2019](#References) and [Ha, 2017](#References), pgpelib also implements the ask-and-tell interface. In the simplest cases, the `ask(...)` method of the solver is used for requesting solutions to evaluate, and the `tell(...)` method is used for sending the solver the evaluation results (fitness values).

Below is our implemented evolution loop.

In [5]:
# Number of iterations
num_iterations = 5000

# At every `report_interval` amount of iterations, show the status:
report_interval = 100

# The main loop of the evolutionary computation
for i in range(1, 1 + num_iterations):
    solutions = pgpe.ask()
    fitnesses = [rastrigin_function(x) for x in solutions]
    pgpe.tell(fitnesses)
    
    if (i % report_interval) == 0:
        print("Iteration:", i, "  cost:", -rastrigin_function(pgpe.center))
    

Iteration: 100   cost: 1617.0593872070312
Iteration: 200   cost: 1410.7705383300781
Iteration: 300   cost: 1190.9596099853516
Iteration: 400   cost: 1256.0968627929688
Iteration: 500   cost: 886.995231628418
Iteration: 600   cost: 1135.7402038574219
Iteration: 700   cost: 834.6457214355469
Iteration: 800   cost: 576.2149047851562
Iteration: 900   cost: 588.3603515625
Iteration: 1000   cost: 502.8448181152344
Iteration: 1100   cost: 489.0048828125
Iteration: 1200   cost: 504.94952392578125
Iteration: 1300   cost: 596.1562194824219
Iteration: 1400   cost: 546.8566284179688
Iteration: 1500   cost: 494.8240661621094
Iteration: 1600   cost: 457.27850341796875
Iteration: 1700   cost: 356.2861328125
Iteration: 1800   cost: 337.39385986328125
Iteration: 1900   cost: 226.965576171875
Iteration: 2000   cost: 167.3988037109375
Iteration: 2100   cost: 168.00189208984375
Iteration: 2200   cost: 61.446533203125
Iteration: 2300   cost: 26.16448974609375
Iteration: 2400   cost: 27.14453125
Iteration: 

---

## References

**Ha, D.** (2017). Evolving Stable Strategies. blog.otoro.net.
Link: http://blog.otoro.net/2017/11/12/evolving-stable-strategies/

**Hansen, N., Akimoto, Y., & Baudis, P.** (2019, February). CMA-ES/pycma on Github. Zenodo, DOI:10.5281/zenodo.2559634. Link: https://github.com/CMA-ES/pycma

**Kingma, D. P. & J. Ba.** (2015).  Adam: A method for stochastic optimization.  In Proceedings of 3rd International Conference on Learning Representations. arXiv link: https://arxiv.org/abs/1412.6980

**Rastrigin, L. A.** (1974). "Systems of extremal control." Mir, Moscow.

**Rudolph, G.** (1990, July). "Globale Optimierung mit parallelen Evolutionsstrategien". Diplomarbeit. Department of Computer Science, University of Dortmund.

**Salimans, T., Ho, J., Chen, X., Sidor, S., & Sutskever, I.** (2017). Evolution strategies as a scalable alternative to reinforcement learning. arXiv preprint arXiv:1703.03864.
arXiv link: https://arxiv.org/abs/1703.03864

**Sehnke, F., Osendorfer, C., Rückstieß, T., Graves, A., Peters, J., & Schmidhuber, J.** (2010). Parameter-exploring policy gradients. Neural Networks, 23(4), 551-559.

**Toklu, N. E., Liskowski, P., & Srivastava, R. K.** (2020, September). ClipUp: A Simple and Powerful Optimizer for Distribution-Based Policy Evolution. In International Conference on Parallel Problem Solving from Nature (pp. 515-527). Springer, Cham. arXiv link: https://arxiv.org/abs/2008.02387